# Veracity Project: Distributed Ledgers and Prolog
## Hayden McAlister - University of Otago 2022

### Running this demonstration
This demonstration is (currently) hosted on [Catalyst Cloud](https://catalystcloud.nz/). However, the IP address associated with this project is floating and could change regularly. If desired, this could be assigned some fixed domain name so this demonstration could be accessed anywhere, anytime.

Otherwise, this demonstration is accessible from GitHub as a docker-compose system. [This repository](TODO) describes how to download, run, and interact with this demonstration locally. 

---

## The infrastructure

In this research project we investigated how distributed ledger technology could be used in a veracity context to verify claims and reach consensus on the state of a system. Other research was done concurrently into using Prolog to describe the state of a system and look at provenance in an organ donation scenario. I built the infrastructure to integrate this Prolog into a jupyter notebook (for accessibility and easy interaction) and expose the blockchain for use.

We agreed that for a proof of concept a small scale, locally hosted blockchain would be most suitable. There is no need to put our tests on a "real" ledger at this stage, but if this project moves forward in this direction it may be useful to put information on a public ledger so it can be verified.

My first task was to find a suitable blockchain technology that could be locally hosted. I found [Hyperledger](https://www.hyperledger.org/) to be very promising, as the opensource projects could be locally hosted and seemed very flexible to cover many situations. I looked in to using Hyperledger [Indy](https://www.hyperledger.org/use/hyperledger-indy), [Aries](https://www.hyperledger.org/use/aries), [Besu](https://www.hyperledger.org/use/besu), and [Ursa](https://www.hyperledger.org/use/ursa) but determined that [Hyperledger Iroha](https://www.hyperledger.org/use/iroha) would be most suitable for my needs. This is mainly due to Iroha having the best documentation of the projects I looked at, and was the easiest to start developing with quickly. However, other projects could absolutely be used and in future local hosting may not even be needed.

I spent several weeks setting up infrastructure using Iroha and Docker. I managed to get four Iroha nodes running and comunicating in separate containers, which is significant as four nodes is the minimum neccisary to detect and correct for a single faulty node (crash fault or byzantine fault). While this demonstration does not look any further into faulty nodes, this infrastructure would support probing in this direction.

We can see these containers running. If running this demonstration locally, you can run 

`docker container ls`

and spot the containers labeled `iroha1` through `iroha4`. These containers are hosting the blockchain nodes and are constantly communicating to see if new transactions have arrived. These nodes are backed by the containers `some-postgres1` through `some-postgres4`. I am sure with enough convincing the postgres containers could be consolidated into a single container with several logins (one for each Iroha node), however this infrastructure was the easiest to develop quickly. 

We can also see what the current state of the blockchain is by running the following command. We will discuss this command later on, but for now just know it will query `iroha1` and get all blocks on the chain, storing them in `logs/blocks.log`.

In [4]:
%python
from IrohaUtils import *
log_all_blocks("blocks.log")

We can see in this file that the blocks are represented in JSON format, and are mostly incomprehensible jargon. A lot of this can be understood by trawling through [the documentation](https://iroha.readthedocs.io/en/main/), however this is time consuming and very dense (I said the documentation was the best of the projects above, I did not claim the documentation was clear). For this reason, I have developed some tools we will discuss soon that make developing and interacting with the Iroha network much easier. 